In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datetime as dt

import music_generator.analysis.preprocessing as pp
from music_generator.analysis.play import play_array

import music_generator.analysis.stft as stft

In [3]:
# https://blog.keras.io/building-autoencoders-in-keras.html

import keras

from keras.layers import Dense, Dropout, PReLU, Input
from keras.models import Model
from keras.optimizers import Adam

from keras.regularizers import l2
from keras.models import load_model

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from keras.layers import Input, Dense, Lambda
from keras.models import Model, Sequential
from keras import backend as K
from keras import metrics
from keras.datasets import mnist, fashion_mnist
from sklearn.preprocessing import MinMaxScaler

from keras.layers import LSTM, Lambda

/home/marcelra/Pycharmprojects/music-generator/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
window = 512
lstm_dim = 64
latent_dim = 8
batch_size = 32

In [5]:
# sampling_rate, data = pp.read_wave_file("../data/Verplichte Kots_v9.wav", 0)
sampling_rate, data = pp.read_wave_file("../data/Dream Theater - Another day-fm7ntyycGbU.wav", 0)
# play_array(data, sampling_rate, range_secs=[17, 31])
# data = data[15*44100:29*44100]

In [6]:
x_train = pp.create_batches(data, window, window)
x_train = np.array(x_train)
x_train = x_train.reshape(-1, window, 1)
x_train = x_train[0:(len(x_train) // batch_size) * batch_size]

In [7]:
def sampling(args):
    z_mean, z_log_var = args
    noise = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim))
    return z_mean + K.exp(z_log_var / 2) * noise

In [8]:
def custom_loss(y_true, y_pred):
    # xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
    mse_loss = window * metrics.mse(y_true, y_pred)
    kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = K.mean(mse_loss + 0.001 * kl_loss)
    return vae_loss

In [10]:
inp = Input(batch_shape=(batch_size, window, 1))
lstm = LSTM(lstm_dim, 
            stateful=True, 
            return_sequences=False,
            batch_input_shape=(batch_size, window, 1))(inp)

inter = Dense(lstm_dim, activation='relu')(lstm)
# inter = lstm

z_mean = Dense(latent_dim)(inter)
z_log_var = Dense(latent_dim)(inter)
z_sampled = Lambda(sampling, (latent_dim,))([z_mean, z_log_var])

decoder = Dense(1024, activation='relu')
decoder_mean = Dense(window, activation='linear')

decoded = decoder(z_sampled)
# decoded = z_sampled
out = decoder_mean(decoded)

model = Model(inp, out)
model.compile('adam', loss=custom_loss)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (32, 512, 1)         0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (32, 64)             16896       input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (32, 64)             4160        lstm_2[0][0]                     
__________________________________________________________________________________________________
dense_2 (Dense)                 (32, 8)              520         dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (D

In [12]:
ii = 0
while True:
    model.fit(x_train, x_train.reshape(-1, window), epochs=30, batch_size=batch_size, verbose=1)
    print("Storing to file...")
    y_pred = model.predict(x_train)
    # result = pp.batches_to_array(y_pred, window)
    result = y_pred.reshape(-1)
    pp.store_wave_file(f'temp_{ii}.wav', 44100, result)    
    ii += 1

Epoch 1/30
24960/24960 [==============================] - 111s 4ms/step - loss: 8.0599
Epoch 2/30
24960/24960 [==============================] - 111s 4ms/step - loss: 7.8382
Epoch 3/30
24960/24960 [==============================] - 111s 4ms/step - loss: 8.0033
Epoch 4/30
24960/24960 [==============================] - 111s 4ms/step - loss: 7.5162
Epoch 5/30
24960/24960 [==============================] - 111s 4ms/step - loss: 7.4103
Epoch 6/30
24960/24960 [==============================] - 112s 4ms/step - loss: 7.2739
Epoch 7/30
24960/24960 [==============================] - 112s 4ms/step - loss: 7.1538
Epoch 8/30
24960/24960 [==============================] - 112s 4ms/step - loss: 6.9805
Epoch 9/30
24960/24960 [==============================] - 112s 5ms/step - loss: 6.7662
Epoch 10/30
24960/24960 [==============================] - 112s 4ms/step - loss: 6.6391
Epoch 11/30
24960/24960 [==============================] - 111s 4ms/step - loss: 6.5681
Epoch 12/30
24960/24960 [================

24960/24960 [==============================] - 112s 4ms/step - loss: 5.5394
Epoch 4/30
24960/24960 [==============================] - 112s 4ms/step - loss: 5.5464
Epoch 5/30
24960/24960 [==============================] - 112s 4ms/step - loss: 5.5309
Epoch 6/30
24960/24960 [==============================] - 112s 4ms/step - loss: 5.5362
Epoch 7/30
24960/24960 [==============================] - 112s 5ms/step - loss: 5.5485
Epoch 8/30
24960/24960 [==============================] - 112s 4ms/step - loss: 5.5331
Epoch 9/30
24960/24960 [==============================] - 112s 4ms/step - loss: 5.5352
Epoch 10/30
24960/24960 [==============================] - 113s 5ms/step - loss: 5.5234
Epoch 11/30
24960/24960 [==============================] - 113s 5ms/step - loss: 5.5203
Epoch 12/30
24960/24960 [==============================] - 112s 5ms/step - loss: 5.5200
Epoch 13/30
24960/24960 [==============================] - 112s 4ms/step - loss: 5.5192
Epoch 14/30
24960/24960 [=========================

KeyboardInterrupt: 

# Deep & heavy model

In [ ]:
encoder = Model(inp, z_mean)


In [ ]:
x_encoded = encoder.predict(x_train)

In [ ]:
x_encoded_backup = x_encoded.copy()

In [ ]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [ ]:
x_encoded = x_encoded_backup[2000:3000]

trace = [go.Scatter3d(x=x_encoded[:, 0], 
                      y=x_encoded[:, 1], 
                      z=np.arange(len(x_encoded)),
                      mode='line',
                        marker=dict(
                            size=3,
                            color=np.arange(len(x_encoded)),                # set color to an array/list of desired values
                            colorscale='Viridis',   # choose a colorscale
                            opacity=0.8
                        ))]

fig = go.Figure(data=trace, layout=go.Layout(height=800, width=1000))
iplot(fig)

In [ ]:
# inp = Input(shape=(window, 1))
# lstm = LSTM(128, return_sequences=False)(inp)
# inter = Dense(64, activation='relu')(lstm)
# inter = Dense(16, activation='relu')(inter)
# inter = Dense(2, activation='relu')(inter)
# inter = Dense(16, activation='relu')(inter)
# inter = Dense(64, activation='relu')(inter)
# out = Dense(window, activation='linear')(inter)

# model = Model(inp, out)
# model.compile('adam', 'mse')
# model.summary()